<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 5px; height: 70px">

# Optimising Hospital Bed Occupancy through Machine Learning
**DSI-41 Group FWSG**: Muhammad Faaiz Khan, Sharifah Nurulhuda, Tan Wei Chiong, Gabriel Tan

### 01_02 Data Collection - Bed Occupancy Rates and merging with other datasets

This section outlines the steps taken to collate and clean various data sources into a single working dataset. These raw data sources are:
1. Daily BOR (Bed Occupancy Rates) statistics between 2018 to 2023, retrieved from the [MOH website](https://www.moh.gov.sg/resources-statistics/healthcare-institution-statistics/beds-occupancy-rate-(bor)) and currently saved as `bed_occupancy_rates.csv`
2. Weather (rainfall) data, which we had previously scraped from the [Meteorological Service Singapore](https://www.weather.gov.sg/home/) website and currently saved as `weather_records.csv`
3. Wet bulb temperature data statistics, retrieved from [Data.gov.sg](https://beta.data.gov.sg/collections/1423/datasets/d_f222c70a7c00c5a5a9d4ec432d67f6e8/view) and currently saved as `wetbulbtemperaturehourly.csv`
3. Road accident injuries (only monthly data available for 2018 to 2022), retrieved from the [Singapore Police Force website](https://www.police.gov.sg/-/media/4E82276DD8944CD798DCB65EEDFDCA7B.ashx) in the _'SPF Traffic Annual 2022' report, Table 3C_. As this data was only available in PDF format, we manually converted it to a csv table which is currently saved as `road_injuries.csv`
4. Averaged daily infectious disease cases (i.e. dengue, acute respiratory tract infections, acute diarrhea), retrieved from [MOH website](https://www-moh-gov-sg-admin.cwp.sg/resources-statistics). The relevant features were manually saved as `weekly-infectious-disease-2018-2023.csv`
5. Public holidays between 2018 to 2023 as outlined by the [Ministry of Manpower website](https://www.mom.gov.sg/employment-practices/public-holidays)

We hypothesise the above features as having cyclical or seasonal patterns that we may see mirrored in fluctuations of NUH's BOR data (our target variable).

### 1. Importing and cleaning Bed Occupancy Rates

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re

In [2]:
bor_df = pd.read_csv('../datasets/bed_occupancy_rates.csv')

In [3]:
bor_df.head()

,Bed Occupancy Rate,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Years,Date,AH,CGH,KTPH,NTFGH,NUH(A),SGH,SKH,TTSH
2,2018,01/01/2018,39.9%,81.3%,96.8%,78.9%,73.5%,74.3%,NaN,93.6%
3,2018,02/01/2018,38.2%,85.8%,100.0%,84.7%,78.4%,83.8%,NaN,93.4%
4,2018,03/01/2018,39.9%,86.4%,99.9%,87.5%,83.1%,88.1%,NaN,92.5%


We see that the data isn't really in a format that we want. Specifically, we want row 1 to be the headers/columns, and the rest of the rows from row 2 onwards to be our actual data. 

In light of that, let us create another dataframe with just those pieces of data.

In [4]:
# headers
bor_cols = bor_df.loc[1,:]

In [5]:
# actual data
bor_data = bor_df.loc[2:,:]

In [6]:
bor_cols.tolist()

['Years', 'Date', 'AH', 'CGH', 'KTPH', 'NTFGH', 'NUH(A)', 'SGH', 'SKH', 'TTSH']

In [7]:
bor_data.to_numpy()

array([['2018', '01/01/2018', '39.9%', ..., '74.3%', nan, '93.6%'],
       ['2018', '02/01/2018', '38.2%', ..., '83.8%', nan, '93.4%'],
       ['2018', '03/01/2018', '39.9%', ..., '88.1%', nan, '92.5%'],
       ...,
       ['2024', '11/01/2024', '83.5%', ..., '87.5%', '88.5%', '97.5%'],
       ['2024', '12/01/2024', '78.4%', ..., '86.2%', '87.9%', '96.5%'],
       ['2024', '13/01/2024', '75.9%', ..., '85.5%', '87.2%', '97.5%']],
      dtype=object)

In [8]:
# creating a new DataFrame from the above arrays
cleaned_bor_df = pd.DataFrame.from_records(bor_data.to_numpy(),
                                           columns=[colname.lower() for colname in bor_cols.tolist()])

In [9]:
cleaned_bor_df

,years,date,ah,cgh,ktph,ntfgh,nuh(a),sgh,skh,ttsh
0,2018,01/01/2018,39.9%,81.3%,96.8%,78.9%,73.5%,74.3%,NaN,93.6%
1,2018,02/01/2018,38.2%,85.8%,100.0%,84.7%,78.4%,83.8%,NaN,93.4%
2,2018,03/01/2018,39.9%,86.4%,99.9%,87.5%,83.1%,88.1%,NaN,92.5%
3,2018,04/01/2018,42.4%,85.5%,100.0%,87.1%,82.6%,87.5%,NaN,91.9%
4,2018,05/01/2018,42.9%,84.4%,100.0%,84.3%,79.8%,84.8%,NaN,90.5%
...,...,...,...,...,...,...,...,...,...,...
2199,2024,09/01/2024,84.2%,85.0%,100.0%,89.3%,89.9%,88.8%,90.4%,99.3%
2200,2024,10/01/2024,86.0%,83.7%,99.8%,89.3%,89.6%,87.5%,90.6%,98.8%
2201,2024,11/01/2024,83.5%,82.3%,98.8%,89.9%,88.4%,87.5%,88.5%,97.5%
2202,2024,12/01/2024,78.4%,83.4%,95.8%,86.1%,87.7%,86.2%,87.9%,96.5%


Now, let us rename some columns. Specifically, the `years` column would be better named `year`, and the `nuh(a)` column (referring to adult beds in NUH) would be better simply named `nuh` instead, as we would only consider adult bed occupancies for now.

In [10]:
cleaned_bor_df.rename(columns={'years': 'year', 'nuh(a)': 'nuh'}, inplace=True)

In [11]:
cleaned_bor_df

,year,date,ah,cgh,ktph,ntfgh,nuh,sgh,skh,ttsh
0,2018,01/01/2018,39.9%,81.3%,96.8%,78.9%,73.5%,74.3%,NaN,93.6%
1,2018,02/01/2018,38.2%,85.8%,100.0%,84.7%,78.4%,83.8%,NaN,93.4%
2,2018,03/01/2018,39.9%,86.4%,99.9%,87.5%,83.1%,88.1%,NaN,92.5%
3,2018,04/01/2018,42.4%,85.5%,100.0%,87.1%,82.6%,87.5%,NaN,91.9%
4,2018,05/01/2018,42.9%,84.4%,100.0%,84.3%,79.8%,84.8%,NaN,90.5%
...,...,...,...,...,...,...,...,...,...,...
2199,2024,09/01/2024,84.2%,85.0%,100.0%,89.3%,89.9%,88.8%,90.4%,99.3%
2200,2024,10/01/2024,86.0%,83.7%,99.8%,89.3%,89.6%,87.5%,90.6%,98.8%
2201,2024,11/01/2024,83.5%,82.3%,98.8%,89.9%,88.4%,87.5%,88.5%,97.5%
2202,2024,12/01/2024,78.4%,83.4%,95.8%,86.1%,87.7%,86.2%,87.9%,96.5%


Next, we convert the date column to datetime.

In [12]:
cleaned_bor_df['date'] = pd.to_datetime(cleaned_bor_df['date'], format='%d/%m/%Y')

In [13]:
cleaned_bor_df

,year,date,ah,cgh,ktph,ntfgh,nuh,sgh,skh,ttsh
0,2018,2018-01-01,39.9%,81.3%,96.8%,78.9%,73.5%,74.3%,NaN,93.6%
1,2018,2018-01-02,38.2%,85.8%,100.0%,84.7%,78.4%,83.8%,NaN,93.4%
2,2018,2018-01-03,39.9%,86.4%,99.9%,87.5%,83.1%,88.1%,NaN,92.5%
3,2018,2018-01-04,42.4%,85.5%,100.0%,87.1%,82.6%,87.5%,NaN,91.9%
4,2018,2018-01-05,42.9%,84.4%,100.0%,84.3%,79.8%,84.8%,NaN,90.5%
...,...,...,...,...,...,...,...,...,...,...
2199,2024,2024-01-09,84.2%,85.0%,100.0%,89.3%,89.9%,88.8%,90.4%,99.3%
2200,2024,2024-01-10,86.0%,83.7%,99.8%,89.3%,89.6%,87.5%,90.6%,98.8%
2201,2024,2024-01-11,83.5%,82.3%,98.8%,89.9%,88.4%,87.5%,88.5%,97.5%
2202,2024,2024-01-12,78.4%,83.4%,95.8%,86.1%,87.7%,86.2%,87.9%,96.5%


Notice that the bed occupancy rates are all in percentages. It would be troublesome to plot out any time series if the values stay as percentages with the % symbol. Thus, we shall remove them.

In [14]:
cleaned_bor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204 entries, 0 to 2203
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   year    2204 non-null   object        
 1   date    2204 non-null   datetime64[ns]
 2   ah      2204 non-null   object        
 3   cgh     2204 non-null   object        
 4   ktph    2204 non-null   object        
 5   ntfgh   2204 non-null   object        
 6   nuh     2204 non-null   object        
 7   sgh     2204 non-null   object        
 8   skh     1981 non-null   object        
 9   ttsh    2204 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 172.3+ KB


In [15]:
cleaned_bor_df.columns.tolist()[2:]

['ah', 'cgh', 'ktph', 'ntfgh', 'nuh', 'sgh', 'skh', 'ttsh']

In [16]:
for col in cleaned_bor_df.columns.tolist()[2:]:
    cleaned_bor_df[col] = [float(str(entry)[:-1]) if type(entry) == str else entry for entry in cleaned_bor_df[col]]

In [17]:
cleaned_bor_df

,year,date,ah,cgh,ktph,ntfgh,nuh,sgh,skh,ttsh
0,2018,2018-01-01,39.9,81.3,96.8,78.9,73.5,74.3,NaN,93.6
1,2018,2018-01-02,38.2,85.8,100.0,84.7,78.4,83.8,NaN,93.4
2,2018,2018-01-03,39.9,86.4,99.9,87.5,83.1,88.1,NaN,92.5
3,2018,2018-01-04,42.4,85.5,100.0,87.1,82.6,87.5,NaN,91.9
4,2018,2018-01-05,42.9,84.4,100.0,84.3,79.8,84.8,NaN,90.5
...,...,...,...,...,...,...,...,...,...,...
2199,2024,2024-01-09,84.2,85.0,100.0,89.3,89.9,88.8,90.4,99.3
2200,2024,2024-01-10,86.0,83.7,99.8,89.3,89.6,87.5,90.6,98.8
2201,2024,2024-01-11,83.5,82.3,98.8,89.9,88.4,87.5,88.5,97.5
2202,2024,2024-01-12,78.4,83.4,95.8,86.1,87.7,86.2,87.9,96.5


We will keep the BOR data for NUH only, since we are focusing exclusively on NUH for this project.

In [18]:
cleaned_bor_df = cleaned_bor_df[['year', 'date', 'nuh']]
cleaned_bor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204 entries, 0 to 2203
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   year    2204 non-null   object        
 1   date    2204 non-null   datetime64[ns]
 2   nuh     2204 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 51.8+ KB


### 2. Importing and cleaning rainfall data

In [19]:
weather_records_df = pd.read_csv('../datasets/weather_records.csv')

In [20]:
weather_records_df.head()

,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,Admiralty,2023.0,1.0,1.0,0.0,0.0,0.0,0.0,26.8,30.1,24.8,10.7,27.8
1,Admiralty,2023.0,1.0,2.0,0.0,0.0,0.0,0.0,27.3,31.5,24.7,13.6,34.3
2,Admiralty,2023.0,1.0,3.0,0.2,0.2,0.2,0.2,27.3,31.7,25.0,11.2,35.6
3,Admiralty,2023.0,1.0,4.0,0.0,0.0,0.0,0.0,26.5,29.3,24.5,13.2,41.5
4,Admiralty,2023.0,1.0,5.0,3.4,3.2,3.4,3.4,27.1,32.2,25.3,8.7,32.4


Let's rename the columns such that each column name:
- Is all lowercase
- Has no non-alphanumeric symbols other than underscores
- Is separated by underscores instead of whitespace

In [21]:
weather_records_df.columns = [re.sub('[^\w]+', '_', re.sub('\(|\)', ' ', str(colname).lower()).strip(' ')) for colname in weather_records_df.columns]

In [22]:
weather_records_df.head()

,station,year,month,day,daily_rainfall_total_mm,highest_30_min_rainfall_mm,highest_60_min_rainfall_mm,highest_120_min_rainfall_mm,mean_temperature_c,maximum_temperature_c,minimum_temperature_c,mean_wind_speed_km_h,max_wind_speed_km_h
0,Admiralty,2023.0,1.0,1.0,0.0,0.0,0.0,0.0,26.8,30.1,24.8,10.7,27.8
1,Admiralty,2023.0,1.0,2.0,0.0,0.0,0.0,0.0,27.3,31.5,24.7,13.6,34.3
2,Admiralty,2023.0,1.0,3.0,0.2,0.2,0.2,0.2,27.3,31.7,25.0,11.2,35.6
3,Admiralty,2023.0,1.0,4.0,0.0,0.0,0.0,0.0,26.5,29.3,24.5,13.2,41.5
4,Admiralty,2023.0,1.0,5.0,3.4,3.2,3.4,3.4,27.1,32.2,25.3,8.7,32.4


In [23]:
weather_records_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116205 entries, 0 to 116204
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   station                      116205 non-null  object 
 1   year                         115780 non-null  float64
 2   month                        115780 non-null  float64
 3   day                          115780 non-null  float64
 4   daily_rainfall_total_mm      116205 non-null  object 
 5   highest_30_min_rainfall_mm   116205 non-null  object 
 6   highest_60_min_rainfall_mm   116205 non-null  object 
 7   highest_120_min_rainfall_mm  116205 non-null  object 
 8   mean_temperature_c           116205 non-null  object 
 9   maximum_temperature_c        116010 non-null  object 
 10  minimum_temperature_c        116010 non-null  object 
 11  mean_wind_speed_km_h         116205 non-null  object 
 12  max_wind_speed_km_h          116008 non-null  object 
dtyp

We replace the `-` entries with null values.

In [24]:
for col in weather_records_df.columns.tolist():
    weather_records_df[col] = weather_records_df.apply(lambda row: np.nan if row[col]=="-" else row[col], axis=1)

weather_records_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116205 entries, 0 to 116204
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   station                      116205 non-null  object 
 1   year                         115780 non-null  float64
 2   month                        115780 non-null  float64
 3   day                          115780 non-null  float64
 4   daily_rainfall_total_mm      112048 non-null  object 
 5   highest_30_min_rainfall_mm   111384 non-null  object 
 6   highest_60_min_rainfall_mm   111285 non-null  object 
 7   highest_120_min_rainfall_mm  111288 non-null  object 
 8   mean_temperature_c           70128 non-null   object 
 9   maximum_temperature_c        71026 non-null   object 
 10  minimum_temperature_c        70998 non-null   object 
 11  mean_wind_speed_km_h         69810 non-null   object 
 12  max_wind_speed_km_h          70535 non-null   object 
dtyp

In [25]:
changi_records_df = weather_records_df[weather_records_df['station'] == 'Changi'].copy()
changi_records_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2191 entries, 16519 to 18709
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   station                      2191 non-null   object 
 1   year                         2191 non-null   float64
 2   month                        2191 non-null   float64
 3   day                          2191 non-null   float64
 4   daily_rainfall_total_mm      2191 non-null   object 
 5   highest_30_min_rainfall_mm   2188 non-null   object 
 6   highest_60_min_rainfall_mm   2186 non-null   object 
 7   highest_120_min_rainfall_mm  2186 non-null   object 
 8   mean_temperature_c           2191 non-null   object 
 9   maximum_temperature_c        2191 non-null   object 
 10  minimum_temperature_c        2191 non-null   object 
 11  mean_wind_speed_km_h         2189 non-null   object 
 12  max_wind_speed_km_h          2183 non-null   object 
dtypes: float64(3), obj

In order to merge the columns we need, we create a `date` column in datetime format using the given year/month/day values.

In [26]:
def get_datetime(row):
    return dt.datetime(int(row['year']), int(row['month']), int(row['day']))

changi_records_df['date'] = changi_records_df.apply(get_datetime, axis=1)
changi_records_df.head()

,station,year,month,day,daily_rainfall_total_mm,highest_30_min_rainfall_mm,highest_60_min_rainfall_mm,highest_120_min_rainfall_mm,mean_temperature_c,maximum_temperature_c,minimum_temperature_c,mean_wind_speed_km_h,max_wind_speed_km_h,date
16519,Changi,2023.0,1.0,1.0,0.0,0.0,0.0,0.0,27.2,30.6,25.3,9.8,31.5,2023-01-01
16520,Changi,2023.0,1.0,2.0,0.0,0.0,0.0,0.0,27.4,31.6,25.1,11.6,38.9,2023-01-02
16521,Changi,2023.0,1.0,3.0,6.0,6.0,6.0,6.0,27.4,31.8,24.4,9.3,37.0,2023-01-03
16522,Changi,2023.0,1.0,4.0,2.6,1.4,1.8,1.8,26.2,28.6,23.9,9.0,37.0,2023-01-04
16523,Changi,2023.0,1.0,5.0,0.2,0.2,0.2,0.2,27.1,31.3,25.2,6.3,27.8,2023-01-05


Lastly, we drop everything in the `changi_records_df` dataframe except for the `date` and `daily_rainfall_total_mm` features. We will also rename `daily_rainfall_total_mm` to `total_rainfall` for brevity.

In [27]:
changi_records_df = changi_records_df[['date', 'daily_rainfall_total_mm']]
changi_records_df = changi_records_df.rename(columns={'daily_rainfall_total_mm': 'total_rainfall'})

changi_records_df.head()

,date,total_rainfall
16519,2023-01-01,0.0
16520,2023-01-02,0.0
16521,2023-01-03,6.0
16522,2023-01-04,2.6
16523,2023-01-05,0.2


### 3. Importing and cleaning wet bulb temperature data

In [28]:
wbt_hourly_df = pd.read_csv('../datasets/wetbulbtemperaturehourly.csv')

In [29]:
wbt_hourly_df.head()

,wbt_date,wbt_time,wet_bulb_temperature
0,1982-01-01,1,24.7
1,1982-01-01,2,24.5
2,1982-01-01,3,24.3
3,1982-01-01,4,24.2
4,1982-01-01,5,24.2


First, we drop the `wbt_time` column, which organizes the data hourly, since all we want is the daily maximum.

In [30]:
wbt_hourly_df.drop(columns=['wbt_time'], inplace=True)

Now that the hours are out of the way, we have a group of different temperature readings indexed by the same day. This invites us to use the `df.groupby` aggregation method to compute the maximum wet bulb temperature for each day.

In [31]:
wbt_daily_df = wbt_hourly_df.groupby('wbt_date').agg('max')
wbt_daily_df.reset_index(inplace=True)

In [32]:
wbt_daily_df.head()

,wbt_date,wet_bulb_temperature
0,1982-01-01,25.5
1,1982-01-02,24.6
2,1982-01-03,25.1
3,1982-01-04,25.5
4,1982-01-05,24.8


In [33]:
wbt_daily_df.rename(columns={'wbt_date': 'date'}, inplace=True)
wbt_daily_df['date'] = pd.to_datetime(wbt_daily_df['date'])
wbt_daily_df.head()

,date,wet_bulb_temperature
0,1982-01-01,25.5
1,1982-01-02,24.6
2,1982-01-03,25.1
3,1982-01-04,25.5
4,1982-01-05,24.8


### 4. Importing and cleaning infectious diseases data

In [34]:
init_diseases_df = pd.read_csv('../datasets/weekly-infectious-disease-2018-2023.csv')

In [35]:
init_diseases_df.tail()

,Year,Epidemiology Wk,Start,End,Dengue (Weekly),DHF (Weekly),Average Dengue + DHF (Daily),Acute Upper Respiratory Tract infections (Daily),Acute Diarrhoea (Daily)
308,2023,48,26/11/2023,2/12/2023,142,2,21,2970,294
309,2023,49,3/12/2023,9/12/2023,183,0,26,3590,285
310,2023,50,10/12/2023,16/12/2023,211,0,30,3475,261
311,2023,51,17/12/2023,23/12/2023,236,0,34,2853,272
312,2023,52,24/12/2023,30/12/2023,275,1,39,2467,324


In [36]:
# Convert the 'date' column to datetime format and set it as the index
init_diseases_df['Start'] = pd.to_datetime(init_diseases_df['Start'], format='%d/%m/%Y')
init_diseases_df = init_diseases_df.set_index('Start')

# Filter to range 2018-2022, then add rows for missing days. The values will be forward filled from existing values.
init_diseases_df = init_diseases_df.reindex(pd.date_range('2018-01-01', '2023-12-31'), method='ffill')

# Filter out unnecessary columns and rename remaining 3 columns
diseases_df = (init_diseases_df[['Average Dengue + DHF (Daily)', 
                                 'Acute Upper Respiratory Tract infections (Daily)', 
                                 'Acute Diarrhoea (Daily)']]
 .rename(columns={'Average Dengue + DHF (Daily)': "dengue", 
                  'Acute Upper Respiratory Tract infections (Daily)': "urti", 
                  'Acute Diarrhoea (Daily)': "diarrhea"}))

# resets index and renames it to date, for easier merging later
diseases_df.reset_index(inplace=True)
diseases_df.rename(columns={'index': 'date'}, inplace=True)

In [37]:
diseases_df.head()

,date,dengue,urti,diarrhea
0,2018-01-01,12,3158,580
1,2018-01-02,12,3158,580
2,2018-01-03,12,3158,580
3,2018-01-04,12,3158,580
4,2018-01-05,12,3158,580


### 5. Importing and cleaning road injury data

In [38]:
# Load data on accidents:
raccidents_df = pd.read_csv('../datasets/road_injuries.csv')

# Check data:
raccidents_df.info()
raccidents_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      180 non-null    object
 1   injuries  180 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.9+ KB


,date,injuries
0,2008-01,992
1,2008-02,790
2,2008-03,951
3,2008-04,876
4,2008-05,867


In [39]:
# Convert the 'date' column to datetime format and set it as the index
raccidents_df['date'] = pd.to_datetime(raccidents_df['date'], format='%Y-%m')
raccidents_df = raccidents_df.set_index('date')

Notably, the `injuries` feature has been recorded as a monthly total. To use this feature for prediction, we will have to derive a daily value for `injuries`.

For this, we will derive the daily traffic accident injuries by obtaining the daily mean, which is obtained from dividing the monthly value by the number of days in the respective month.

In [40]:
# Dividing each value in 'injuries' by the number of days in the month for the daily mean
raccidents_df['injuries'] = (raccidents_df['injuries'] / raccidents_df.index.days_in_month).astype(int)
raccidents_df

,injuries
date,
2008-01-01,32
2008-02-01,27
2008-03-01,30
2008-04-01,29
2008-05-01,27
...,...
2022-08-01,23
2022-09-01,24
2022-10-01,23


Here we introduce additional rows for the missing days of the month.
The missing 'injuries' values are forward filled from the existing value for each month.

In [41]:
raccidents_df = raccidents_df.reindex(pd.date_range('2018-01-01', '2022-12-31'), method='ffill')
raccidents_df = raccidents_df.reset_index()
raccidents_df.rename(columns={'index': 'date'}, inplace=True)

#### Merging the datasets

In [42]:
final_df = cleaned_bor_df.copy()

for data in [changi_records_df, wbt_daily_df, diseases_df, raccidents_df]:
    final_df = pd.merge(final_df, data, on='date', how='left')

final_df.head()

,year,date,nuh,total_rainfall,wet_bulb_temperature,dengue,urti,diarrhea,injuries
0,2018,2018-01-01,73.5,32.2,25.2,12.0,3158.0,580.0,28.0
1,2018,2018-01-02,78.4,0.8,25.8,12.0,3158.0,580.0,28.0
2,2018,2018-01-03,83.1,2.0,26.5,12.0,3158.0,580.0,28.0
3,2018,2018-01-04,82.6,0.2,25.6,12.0,3158.0,580.0,28.0
4,2018,2018-01-05,79.8,1.0,25.4,12.0,3158.0,580.0,28.0


### 6. Engineering additional features for 'day', 'month' and other public holidays

In [43]:
# Convert 'date' column to datetime
final_df['date'] = pd.to_datetime(final_df['date'], format='%d/%m/%Y')

# Set index to be the 'date' column
final_df = final_df.set_index('date')

# Ascending sort on the index: 'date' from earliest to latest
final_df = final_df.sort_index()
final_df = final_df[final_df.index.year.isin([2018,2019,2020,2021,2022,2023])]

In [44]:
# # Create new columns for day and month from time series
final_df['day'] = final_df.index.strftime('%a')
final_df['month'] = final_df.index.strftime('%b')

In [45]:
public_hols_2018_2023 = [dt.datetime(2018,1,1),
                         dt.datetime(2018,2,16),
                         dt.datetime(2018,2,17),
                         dt.datetime(2018,3,30),
                         dt.datetime(2018,5,1),
                         dt.datetime(2018,5,29),
                         dt.datetime(2018,6,15),
                         dt.datetime(2018,8,9),
                         dt.datetime(2018,8,22),
                         dt.datetime(2018,11,6),
                         dt.datetime(2018,12,25),
                         dt.datetime(2019,1,1),
                         dt.datetime(2019,2,5),
                         dt.datetime(2019,2,6),
                         dt.datetime(2019,4,19),
                         dt.datetime(2019,5,1),
                         dt.datetime(2019,5,19),
                         dt.datetime(2019,5,20),
                         dt.datetime(2019,6,5),
                         dt.datetime(2019,8,9),
                         dt.datetime(2019,8,11),
                         dt.datetime(2019,8,12),
                         dt.datetime(2019,10,27),
                         dt.datetime(2019,10,28),
                         dt.datetime(2019,12,25),
                         dt.datetime(2020,1,1),
                         dt.datetime(2020,1,25),
                         dt.datetime(2020,1,26),
                         dt.datetime(2020,1,27),
                         dt.datetime(2020,4,10),
                         dt.datetime(2020,5,1),
                         dt.datetime(2020,5,7),
                         dt.datetime(2020,5,24),
                         dt.datetime(2020,5,25),
                         dt.datetime(2020,7,31),
                         dt.datetime(2020,8,9),
                         dt.datetime(2020,8,10),
                         dt.datetime(2020,11,14),
                         dt.datetime(2020,12,25),
                         dt.datetime(2021,1,1),
                         dt.datetime(2021,2,12),
                         dt.datetime(2021,2,13),
                         dt.datetime(2021,4,2),
                         dt.datetime(2021,5,1),
                         dt.datetime(2021,5,13),
                         dt.datetime(2021,5,26),
                         dt.datetime(2021,7,20),
                         dt.datetime(2021,8,9),
                         dt.datetime(2021,11,4),
                         dt.datetime(2021,12,25),
                         dt.datetime(2022,1,1),
                         dt.datetime(2022,2,1),
                         dt.datetime(2022,2,2),
                         dt.datetime(2022,4,15),
                         dt.datetime(2022,5,1),
                         dt.datetime(2022,5,2),
                         dt.datetime(2022,5,3),
                         dt.datetime(2022,5,15),
                         dt.datetime(2022,5,16),
                         dt.datetime(2022,7,10),
                         dt.datetime(2022,7,11),
                         dt.datetime(2022,8,9),
                         dt.datetime(2022,10,24),
                         dt.datetime(2022,12,25),
                         dt.datetime(2022,12,26),
                         dt.datetime(2023,1,1),
                         dt.datetime(2023,1,2),
                         dt.datetime(2023,1,22),
                         dt.datetime(2023,1,23),
                         dt.datetime(2023,1,24),
                         dt.datetime(2023,4,7),
                         dt.datetime(2023,4,22),
                         dt.datetime(2023,5,1),
                         dt.datetime(2023,6,2),
                         dt.datetime(2023,6,29),
                         dt.datetime(2023,8,9),
                         dt.datetime(2023,9,1),
                         dt.datetime(2023,11,12),
                         dt.datetime(2023,11,13),
                         dt.datetime(2023,12,25)]

In [46]:
def is_holiday(row):
    return int(row.name in public_hols_2018_2023)

In [47]:
final_df['is_holiday'] = final_df.apply(is_holiday, axis=1)

In [48]:
# check specific rows with known holiday dates
final_df.iloc[950:955,:]

,year,nuh,total_rainfall,wet_bulb_temperature,dengue,urti,diarrhea,injuries,day,month,is_holiday
date,,,,,,,,,,,
2020-08-08,2020,81.9,0.0,26.9,238.0,780.0,336.0,16.0,Sat,Aug,0
2020-08-09,2020,75.3,0.0,27.3,191.0,710.0,362.0,16.0,Sun,Aug,1
2020-08-10,2020,80.2,0.0,27.0,191.0,710.0,362.0,16.0,Mon,Aug,1
2020-08-11,2020,85.4,0.0,26.8,191.0,710.0,362.0,16.0,Tue,Aug,0
2020-08-12,2020,85.4,0.0,26.5,191.0,710.0,362.0,16.0,Wed,Aug,0


#### Exporting the cleaned dataset

We will save the working dataset for further EDA:

In [49]:
# Export the final DataFrame for bed occupancy rates to a CSV file
final_df.to_csv('../datasets/df_bor.csv', index=True, index_label='date')